In [2]:
import pandas as pd

df = pd.read_json('math12k_baseline_results.jsonl', lines=True)

In [ ]:
import pandas as pd

df2 = pd.read_json('outputs/lora_sft/lora-full-5e-05-16/', lines=True)

In [7]:
df.shape

(1319, 5)

In [11]:
df["raw_score_result"].iloc[0]

{'format_reward': 1.0, 'answer_reward': 0.0, 'reward': 0.0}

In [8]:
sum(df["raw_score_result"].apply(lambda x: x["format_reward"]))

236.0

In [ ]:
df_filtered = df[
    (df["raw_score_result"].str["format_reward"] == 1.0) & 
    (df["raw_score_result"].str["answer_reward"] == 0.0)
]

In [58]:
df_filtered.shape

(206, 5)

In [60]:
df_filtered[df_filtered["generation"].apply(lambda x: "<think>" in x)].iloc[0]["generation"]

"I know that Raymond has 40 jewels.</think> <think>Aaron has 5 more jewels than half of Raymond's jewels. [/think]\n</thim> $$\\frac{1}{2}*40+5 = 20+5 = 25$$<lead>Aaron has 25 jewels.</lead>\n</think> <think>Aaron has 2 fewer jewels than Siobhan.</think>\n</thim> $$25-2 = 23$$<lead>Siobhan has 23 jewels.</lead>\n</think> <answer>Siobhan has 23 jewels.</answer>"

In [66]:
gsm8k_train = pd.read_json('../data/gsm8k/train.jsonl', lines=True)

In [67]:
gsm8k_train.shape

(7473, 2)

In [14]:
def load_gsm8k(path: str):
    data = []
    with open(path, "r") as f:
        for line in f:
            data.append(json.loads(line))
    return data

In [15]:
import json
data = load_gsm8k("../data/gsm8k/train.jsonl")

In [4]:
len(data)

7473

In [15]:
example = data[0]
example

{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
 'answer': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18'}

In [16]:
example["answer"].split("####")[0].strip()

'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.'

In [17]:
cnt = 0
for i in data:
    if "####" not in i["answer"]:
        cnt+=1

In [16]:
def load_gsm8k(path: str):
    data = []
    with open(path, "r") as f:
        for line in f:
            data.append(json.loads(line))
    return data


def format_gsm8k_example(example, prompt_template):
    """
    Format a GSM8K example using the provided prompt template.
    
    Args:
        example: Dictionary with 'question' and 'answer' keys
        prompt_template: Template string with {question} placeholder
        
    Returns:
        Dictionary with 'prompt', 'response', and 'original_answer' keys
    """
    question = example["question"]
    full_answer = example["answer"]
    
    # Extract reasoning process and final answer
    reasoning = full_answer.split("####")[0].strip()
    answer = full_answer.split("####")[-1].strip()
    
    # Format the prompt using the template 
    prompt = prompt_template.format(question=question)
    
    # Build the response: reasoning + closing tags + answer
    response = f"<think> {reasoning} </think> <answer> {answer} </answer>"
    
    return {
        "prompt": prompt,
        "response": response,
        "reasoning": reasoning,
        "original_answer": answer
    }

In [17]:
import json
with open("prompts/r1_zero.prompt", "r") as f:
    prompt_template = f.read().strip()

train_raw = load_gsm8k("../data/gsm8k/test.jsonl")
formatted_train = [format_gsm8k_example(ex, prompt_template) for ex in train_raw]

In [19]:
formatted_train[1]["reasoning"]

'It takes 2/2=<<2/2=1>>1 bolt of white fiber\nSo the total amount of fabric is 2+1=<<2+1=3>>3 bolts of fabric'

In [22]:
print("hu")

hu


In [5]:
tst = "q_proj,k_proj,v_proj,o_proj,gate_proj,up_proj,down_proj"

In [6]:
[m.strip() for m in tst.split(",") if m.strip()]

['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']